In [1]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

from src.modules.probabilities import Probabilities
import modules.generator as g
from modules.transition import Transition
from src.modules.probabilities import Probabilities
from src.modules.app import App
from src.modules import constants as con


In [ ]:
# (1) Derive Probabilties for transition


np.random.seed(14)
prob = Probabilities()

p = prob.getProbabilities(1*con.tau*60)

#sum(p["p"]) # == 1 !


In [ ]:
from src.modules import constants as con
# Construct ex_info data frame for all ts
df_p = pd.DataFrame()

for t in np.arange(0, 1440, con.tau*60):
    df_p = df_p.append(prob.getProbabilities(t))
    
df_p.tail()

In [ ]:
# (2) Derive state space


import time
start = time.process_time()
# your code here    
sts = g.constructStates()
print(time.process_time() - start)
sts.describe()

In [ ]:
sts.head()
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 2) & (sts["B_L"] >= 40) & (sts["D"] == 0)]) ##120
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 10) & (sts["B_L"] >= 40)]) ##120

In [ ]:
# (3) Derive decision space
from tqdm import tqdm
dec = {}

for s in tqdm(sts["obj"].head(1)):
    dec[s.getKey()] = g.constructDecisions(s)

#print(sts.iloc[0])
#print(dec)
print(pd.DataFrame([x.__str__() for x in dec[sts.loc[0, "obj"].getKey()]]))

In [ ]:
# Perform transitions



np.random.seed(14)

prob = Probabilities()
ex_inf = prob.getProbabilities(300).loc[0,:]
t = Transition(None, sts.loc[0, "obj"], dec[sts.loc[0,"obj"].getKey()][-1], ex_inf["p"], ex_inf[0:3])


print(sts.loc[0, "obj"])
print(dec[sts.loc[0,"obj"].getKey()][-1])
print(ex_inf)
print()
print(t.get_s_d().__str__())

In [8]:
a = App()
df = a.valueIteration()

984it [00:00, 140366.46it/s]
100%|██████████| 984/984 [00:07<00:00, 133.19it/s]


In [11]:
df.loc[(df["t"] == 0) & (df["s_obj"].apply(lambda x: x.get_B_L() == 4.2)) & (df["d_obj"].apply(lambda d: d.get_x_V2G() > 1.0)),~df.columns.isin(['s_obj', 'd_obj', "tr_obj"])]
df.loc[(df["s_obj"].apply(lambda x: x.get_t() == 2)),~df.columns.isin(['d_obj', "tr_obj"])]

,s_key,s_obj,d_key,t,trpstrt,trpln,prc,p,s_d_key
301104,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]","(0.0,0.0,0)",2,y,0,0.1,7.27731e-06,"(3,4.0,0,0,0.1,0.1)"
301105,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]","(0.0,0.0,0)",2,y,0,0.0,4.17555e-06,"(3,4.0,0,0,0.0,0.0)"
301106,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]","(0.0,0.0,0)",2,y,1,0.1,2.18319e-05,"(3,4.0,0,1,0.1,0.1)"
301107,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]","(0.0,0.0,0)",2,y,1,0.0,1.25266e-05,"(3,4.0,0,1,0.0,0.0)"
301108,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]","(0.0,0.0,0)",2,y,2,0.1,1.69804e-05,"(3,4.0,0,2,0.1,0.1)"
...,...,...,...,...,...,...,...,...,...
328979,"(2,5.0,0,0,0.1,0.1)","State t=2[(5.0,0,0,0.1,0.1) - True - (0, 0)]","(0.0,2.2,0)",2,y,38,0.0,0,"(3,2.8,0,38,0.0,0.0)"
328980,"(2,5.0,0,0,0.1,0.1)","State t=2[(5.0,0,0,0.1,0.1) - True - (0, 0)]","(0.0,2.2,0)",2,y,39,0.1,0,"(3,2.8,0,39,0.1,0.1)"
328981,"(2,5.0,0,0,0.1,0.1)","State t=2[(5.0,0,0,0.1,0.1) - True - (0, 0)]","(0.0,2.2,0)",2,y,39,0.0,0,"(3,2.8,0,39,0.0,0.0)"
328982,"(2,5.0,0,0,0.1,0.1)","State t=2[(5.0,0,0,0.1,0.1) - True - (0, 0)]","(0.0,2.2,0)",2,n,0,0.1,0.635143,"(3,2.8,0,0,0.1,0.1)"


In [9]:
a.df_states[a.df_states["s_obj"].apply(lambda s: (s.get_t() == 2))]

,s_key,s_obj
960,"(2,4.0,0,0,0.0,0.0)","State t=2[(4.0,0,0,0.0,0.0) - True - (0, 0)]"
961,"(2,4.0,0,0,0.0,0.1)","State t=2[(4.0,0,0,0.0,0.1) - True - (0, 0)]"
962,"(2,4.0,0,0,0.1,0.0)","State t=2[(4.0,0,0,0.1,0.0) - True - (0, 0)]"
963,"(2,4.0,0,0,0.1,0.1)","State t=2[(4.0,0,0,0.1,0.1) - True - (0, 0)]"
964,"(2,4.2,0,0,0.0,0.0)","State t=2[(4.2,0,0,0.0,0.0) - True - (0, 0)]"
965,"(2,4.2,0,0,0.0,0.1)","State t=2[(4.2,0,0,0.0,0.1) - True - (0, 0)]"
966,"(2,4.2,0,0,0.1,0.0)","State t=2[(4.2,0,0,0.1,0.0) - True - (0, 0)]"
967,"(2,4.2,0,0,0.1,0.1)","State t=2[(4.2,0,0,0.1,0.1) - True - (0, 0)]"
968,"(2,4.4,0,0,0.0,0.0)","State t=2[(4.4,0,0,0.0,0.0) - True - (0, 0)]"
969,"(2,4.4,0,0,0.0,0.1)","State t=2[(4.4,0,0,0.0,0.1) - True - (0, 0)]"
